In [2]:
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


# Importing Data

In [3]:
BBB_Train = pd.read_csv("BBB_LSTM_Input_Train.csv", index_col=0)
DDD_Train = pd.read_csv("DDD_LSTM_Input_Train.csv", index_col=0)
FFF_Train = pd.read_csv("FFF_LSTM_Input_Train.csv", index_col=0)

BBB_Test = pd.read_csv("BBB_LSTM_Input_Test.csv", index_col=0)
DDD_Test = pd.read_csv("DDD_LSTM_Input_Test.csv", index_col=0)
FFF_Test = pd.read_csv("FFF_LSTM_Input_Test.csv", index_col=0)

# Transforming Data

#### Normalize variables for every course  per semester

In [3]:
BBB_Train.head()

,id_student,code_presentation,Decile,Avg_Clicks,Asgmts,Avg_Asgmt_Score,final_result
1,1008675,2013B,0,2.181818,0,0.000000,Pass
2,1008675,2013B,1,12.391304,1,70.000000,Pass
4,1008675,2013B,2,12.875000,1,65.000000,Pass
5,1008675,2013B,3,7.250000,1,56.666667,Pass
6,1008675,2013B,4,5.166667,1,55.250000,Pass


In [5]:
from sklearn.preprocessing import MinMaxScaler

In [6]:
Semesters_Train = BBB_Train.code_presentation.unique()
dfs_Train = list([BBB_Train, DDD_Train, FFF_Train])
Semesters_Test = BBB_Test.code_presentation.unique()
dfs_Test = list([BBB_Test, DDD_Test, FFF_Test])

In [7]:
scaler = MinMaxScaler(feature_range=(0,1))

for i in range(0,len(dfs_Train)):
    for j in range(0,len(Semesters_Train)):
        scaler.fit(dfs_Train[i].loc[dfs_Train[i]["code_presentation"]==Semesters_Train[j], "Avg_Clicks":"Avg_Asgmt_Score"])
        dfs_Train[i].loc[dfs_Train[i]["code_presentation"]==Semesters_Train[j], "Avg_Clicks":"Avg_Asgmt_Score"] = scaler.transform(dfs_Train[i].loc[dfs_Train[i]["code_presentation"]==Semesters_Train[j], "Avg_Clicks":"Avg_Asgmt_Score"])


for i in range(0,len(dfs_Test)):
    for j in range(0,len(Semesters_Test)):
        scaler.fit(dfs_Test[i].loc[dfs_Test[i]["code_presentation"]==Semesters_Test[j], "Avg_Clicks":"Avg_Asgmt_Score"])
        dfs_Test[i].loc[dfs_Test[i]["code_presentation"]==Semesters_Test[j], "Avg_Clicks":"Avg_Asgmt_Score"] = scaler.transform(dfs_Test[i].loc[dfs_Test[i]["code_presentation"]==Semesters_Test[j], "Avg_Clicks":"Avg_Asgmt_Score"])

In [8]:
BBB_Train.head()

,id_student,code_presentation,Decile,Avg_Clicks,Asgmts,Avg_Asgmt_Score,final_result
1,1008675,2013B,0,0.019416,0.000000,0.000000,Pass
2,1008675,2013B,1,0.110267,0.142857,0.700000,Pass
4,1008675,2013B,2,0.114572,0.142857,0.650000,Pass
5,1008675,2013B,3,0.064516,0.142857,0.566667,Pass
6,1008675,2013B,4,0.045977,0.142857,0.552500,Pass


#### Make a 1D array of target labels (pass / fail)

In [9]:
BBB_Train_Targets = BBB_Train["final_result"][0::11]
DDD_Train_Targets = DDD_Train["final_result"][0::11]
FFF_Train_Targets = FFF_Train["final_result"][0::11]

BBB_Test_Targets = BBB_Test["final_result"][0::11]
DDD_Test_Targets = DDD_Test["final_result"][0::11]
FFF_Test_Targets = FFF_Test["final_result"][0::11]

In [10]:
BBB_Train_Targets_1D = np.array(BBB_Train_Targets)
DDD_Train_Targets_1D = np.array(DDD_Train_Targets)
FFF_Train_Targets_1D = np.array(FFF_Train_Targets)

BBB_Test_Targets_1D = np.array(BBB_Test_Targets)
DDD_Test_Targets_1D = np.array(DDD_Test_Targets)
FFF_Test_Targets_1D = np.array(FFF_Test_Targets)


#### Make binary target labels in which "distinction" is considered "pass"

In [11]:
BBB_Train_Targets_1D[BBB_Train_Targets_1D == "Fail"] = 0
BBB_Train_Targets_1D[BBB_Train_Targets_1D == "Pass"] = 1
BBB_Train_Targets_1D[BBB_Train_Targets_1D == "Distinction"] = 1

DDD_Train_Targets_1D[DDD_Train_Targets_1D == "Fail"] = 0
DDD_Train_Targets_1D[DDD_Train_Targets_1D == "Pass"] = 1
DDD_Train_Targets_1D[DDD_Train_Targets_1D == "Distinction"] = 1

FFF_Train_Targets_1D[FFF_Train_Targets_1D == "Fail"] = 0
FFF_Train_Targets_1D[FFF_Train_Targets_1D == "Pass"] = 1
FFF_Train_Targets_1D[FFF_Train_Targets_1D == "Distinction"] = 1

BBB_Test_Targets_1D[BBB_Test_Targets_1D == "Fail"] = 0
BBB_Test_Targets_1D[BBB_Test_Targets_1D == "Pass"] = 1
BBB_Test_Targets_1D[BBB_Test_Targets_1D == "Distinction"] = 1

DDD_Test_Targets_1D[DDD_Test_Targets_1D == "Fail"] = 0
DDD_Test_Targets_1D[DDD_Test_Targets_1D == "Pass"] = 1
DDD_Test_Targets_1D[DDD_Test_Targets_1D == "Distinction"] = 1

FFF_Test_Targets_1D[FFF_Test_Targets_1D == "Fail"] = 0
FFF_Test_Targets_1D[FFF_Test_Targets_1D == "Pass"] = 1
FFF_Test_Targets_1D[FFF_Test_Targets_1D == "Distinction"] = 1

In [12]:
BBB_Train_Targets_1D = BBB_Train_Targets_1D.astype('int64')
DDD_Train_Targets_1D = DDD_Train_Targets_1D.astype('int64')
FFF_Train_Targets_1D = FFF_Train_Targets_1D.astype('int64')

BBB_Test_Targets_1D = BBB_Test_Targets_1D.astype('int64')
DDD_Test_Targets_1D = DDD_Test_Targets_1D.astype('int64')
FFF_Test_Targets_1D = FFF_Test_Targets_1D.astype('int64')

In [13]:
print(FFF_Train_Targets_1D.shape)

(3849,)


In [14]:
FFF_Train_Targets_1D

array([0, 1, 0, ..., 1, 1, 0], dtype=int64)

#### Keep only the relevant variables to use as input for the LSTM model (cut out course, semester, id and decile)

In [15]:
BBB_Train_df = BBB_Train.iloc[:,3:6]
DDD_Train_df = DDD_Train.iloc[:,3:6]
FFF_Train_df = FFF_Train.iloc[:,3:6]

BBB_Test_df = BBB_Test.iloc[:,3:6]
DDD_Test_df = DDD_Test.iloc[:,3:6]
FFF_Test_df = FFF_Test.iloc[:,3:6]

In [16]:
FFF_Train_df.head(n=20)

,Avg_Clicks,Asgmts,Avg_Asgmt_Score
104974,0.010130,0.000,0.000000
104975,0.109139,0.125,0.700000
104977,0.003323,0.000,0.700000
104978,0.003323,0.000,0.700000
104979,0.032648,0.000,0.700000
104980,0.005669,0.125,0.490000
104981,0.000000,0.000,0.490000
104982,0.000000,0.000,0.490000
104983,0.000000,0.000,0.490000
104984,0.000000,0.000,0.490000


Number of students in the courses across all 4 semesters

In [17]:
print("BBB Train\n")
print(BBB_Train_df.count()/11,"\n")

print("DDD Train\n")
print(DDD_Train_df.count()/11,"\n")

print("FFF Train\n")
print(FFF_Train_df.count()/11,"\n")

print("BBB Test\n")
print(BBB_Test_df.count()/11,"\n")

print("DDD Test\n")
print(DDD_Test_df.count()/11,"\n")

print("FFF Test\n")
print(FFF_Test_df.count()/11,"\n")

BBB Train

Avg_Clicks         3978.0
Asgmts             3978.0
Avg_Asgmt_Score    3978.0
dtype: float64 

DDD Train

Avg_Clicks         2866.0
Asgmts             2866.0
Avg_Asgmt_Score    2866.0
dtype: float64 

FFF Train

Avg_Clicks         3849.0
Asgmts             3849.0
Avg_Asgmt_Score    3849.0
dtype: float64 

BBB Test

Avg_Clicks         1543.0
Asgmts             1543.0
Avg_Asgmt_Score    1543.0
dtype: float64 

DDD Test

Avg_Clicks         1156.0
Asgmts             1156.0
Avg_Asgmt_Score    1156.0
dtype: float64 

FFF Test

Avg_Clicks         1510.0
Asgmts             1510.0
Avg_Asgmt_Score    1510.0
dtype: float64 



#### Convert dataframes into 2D arrays

In [18]:
BBB_Train_2D = BBB_Train_df.values
DDD_Train_2D = DDD_Train_df.values
FFF_Train_2D = FFF_Train_df.values

BBB_Test_2D = BBB_Test_df.values
DDD_Test_2D = DDD_Test_df.values
FFF_Test_2D = FFF_Test_df.values

#### Reshape 2D arrays to 3D arrays
Format : samples (students), timesteps (deciles), features (clicks & assignments)

In [19]:
BBB_Train_3D = BBB_Train_2D.reshape(int(BBB_Train_df["Avg_Clicks"].count()/11),11,len(BBB_Train_df.columns))
DDD_Train_3D = DDD_Train_2D.reshape(int(DDD_Train_df["Avg_Clicks"].count()/11),11,len(DDD_Train_df.columns))
FFF_Train_3D = FFF_Train_2D.reshape(int(FFF_Train_df["Avg_Clicks"].count()/11),11,len(FFF_Train_df.columns))

BBB_Test_3D = BBB_Test_2D.reshape(int(BBB_Test_df["Avg_Clicks"].count()/11),11,len(BBB_Test_df.columns))
DDD_Test_3D = DDD_Test_2D.reshape(int(DDD_Test_df["Avg_Clicks"].count()/11),11,len(DDD_Test_df.columns))
FFF_Test_3D = FFF_Test_2D.reshape(int(FFF_Test_df["Avg_Clicks"].count()/11),11,len(FFF_Test_df.columns))


In [20]:
print(FFF_Train_3D.shape)

(3849, 11, 3)


#### Exclude timestep(s) in sequences to be able to build the models for multiple points in time

In [21]:
BBB_Train_3D_Decile9 = np.delete(BBB_Train_3D,-1,axis=1)
BBB_Train_3D_Decile8 = np.delete(BBB_Train_3D,np.s_[-2:],axis=1)
BBB_Train_3D_Decile7 = np.delete(BBB_Train_3D,np.s_[-3:],axis=1)
BBB_Train_3D_Decile6 = np.delete(BBB_Train_3D,np.s_[-4:],axis=1)
BBB_Train_3D_Decile5 = np.delete(BBB_Train_3D,np.s_[-5:],axis=1)
BBB_Train_3D_Decile4 = np.delete(BBB_Train_3D,np.s_[-6:],axis=1)
BBB_Train_3D_Decile3 = np.delete(BBB_Train_3D,np.s_[-7:],axis=1)
BBB_Train_3D_Decile2 = np.delete(BBB_Train_3D,np.s_[-8:],axis=1)
BBB_Train_3D_Decile1 = np.delete(BBB_Train_3D,np.s_[-9:],axis=1)
BBB_Train_3D_Decile0 = np.delete(BBB_Train_3D,np.s_[-10:],axis=1)

BBB_Test_3D_Decile9 = np.delete(BBB_Test_3D,-1,axis=1)
BBB_Test_3D_Decile8 = np.delete(BBB_Test_3D,np.s_[-2:],axis=1)
BBB_Test_3D_Decile7 = np.delete(BBB_Test_3D,np.s_[-3:],axis=1)
BBB_Test_3D_Decile6 = np.delete(BBB_Test_3D,np.s_[-4:],axis=1)
BBB_Test_3D_Decile5 = np.delete(BBB_Test_3D,np.s_[-5:],axis=1)
BBB_Test_3D_Decile4 = np.delete(BBB_Test_3D,np.s_[-6:],axis=1)
BBB_Test_3D_Decile3 = np.delete(BBB_Test_3D,np.s_[-7:],axis=1)
BBB_Test_3D_Decile2 = np.delete(BBB_Test_3D,np.s_[-8:],axis=1)
BBB_Test_3D_Decile1 = np.delete(BBB_Test_3D,np.s_[-9:],axis=1)
BBB_Test_3D_Decile0 = np.delete(BBB_Test_3D,np.s_[-10:],axis=1)

In [22]:
DDD_Train_3D_Decile9 = np.delete(DDD_Train_3D,-1,axis=1)
DDD_Train_3D_Decile8 = np.delete(DDD_Train_3D,np.s_[-2:],axis=1)
DDD_Train_3D_Decile7 = np.delete(DDD_Train_3D,np.s_[-3:],axis=1)
DDD_Train_3D_Decile6 = np.delete(DDD_Train_3D,np.s_[-4:],axis=1)
DDD_Train_3D_Decile5 = np.delete(DDD_Train_3D,np.s_[-5:],axis=1)
DDD_Train_3D_Decile4 = np.delete(DDD_Train_3D,np.s_[-6:],axis=1)
DDD_Train_3D_Decile3 = np.delete(DDD_Train_3D,np.s_[-7:],axis=1)
DDD_Train_3D_Decile2 = np.delete(DDD_Train_3D,np.s_[-8:],axis=1)
DDD_Train_3D_Decile1 = np.delete(DDD_Train_3D,np.s_[-9:],axis=1)
DDD_Train_3D_Decile0 = np.delete(DDD_Train_3D,np.s_[-10:],axis=1)

DDD_Test_3D_Decile9 = np.delete(DDD_Test_3D,-1,axis=1)
DDD_Test_3D_Decile8 = np.delete(DDD_Test_3D,np.s_[-2:],axis=1)
DDD_Test_3D_Decile7 = np.delete(DDD_Test_3D,np.s_[-3:],axis=1)
DDD_Test_3D_Decile6 = np.delete(DDD_Test_3D,np.s_[-4:],axis=1)
DDD_Test_3D_Decile5 = np.delete(DDD_Test_3D,np.s_[-5:],axis=1)
DDD_Test_3D_Decile4 = np.delete(DDD_Test_3D,np.s_[-6:],axis=1)
DDD_Test_3D_Decile3 = np.delete(DDD_Test_3D,np.s_[-7:],axis=1)
DDD_Test_3D_Decile2 = np.delete(DDD_Test_3D,np.s_[-8:],axis=1)
DDD_Test_3D_Decile1 = np.delete(DDD_Test_3D,np.s_[-9:],axis=1)
DDD_Test_3D_Decile0 = np.delete(DDD_Test_3D,np.s_[-10:],axis=1)

In [23]:
FFF_Train_3D_Decile9 = np.delete(FFF_Train_3D,-1,axis=1)
FFF_Train_3D_Decile8 = np.delete(FFF_Train_3D,np.s_[-2:],axis=1)
FFF_Train_3D_Decile7 = np.delete(FFF_Train_3D,np.s_[-3:],axis=1)
FFF_Train_3D_Decile6 = np.delete(FFF_Train_3D,np.s_[-4:],axis=1)
FFF_Train_3D_Decile5 = np.delete(FFF_Train_3D,np.s_[-5:],axis=1)
FFF_Train_3D_Decile4 = np.delete(FFF_Train_3D,np.s_[-6:],axis=1)
FFF_Train_3D_Decile3 = np.delete(FFF_Train_3D,np.s_[-7:],axis=1)
FFF_Train_3D_Decile2 = np.delete(FFF_Train_3D,np.s_[-8:],axis=1)
FFF_Train_3D_Decile1 = np.delete(FFF_Train_3D,np.s_[-9:],axis=1)
FFF_Train_3D_Decile0 = np.delete(FFF_Train_3D,np.s_[-10:],axis=1)

FFF_Test_3D_Decile9 = np.delete(FFF_Test_3D,-1,axis=1)
FFF_Test_3D_Decile8 = np.delete(FFF_Test_3D,np.s_[-2:],axis=1)
FFF_Test_3D_Decile7 = np.delete(FFF_Test_3D,np.s_[-3:],axis=1)
FFF_Test_3D_Decile6 = np.delete(FFF_Test_3D,np.s_[-4:],axis=1)
FFF_Test_3D_Decile5 = np.delete(FFF_Test_3D,np.s_[-5:],axis=1)
FFF_Test_3D_Decile4 = np.delete(FFF_Test_3D,np.s_[-6:],axis=1)
FFF_Test_3D_Decile3 = np.delete(FFF_Test_3D,np.s_[-7:],axis=1)
FFF_Test_3D_Decile2 = np.delete(FFF_Test_3D,np.s_[-8:],axis=1)
FFF_Test_3D_Decile1 = np.delete(FFF_Test_3D,np.s_[-9:],axis=1)
FFF_Test_3D_Decile0 = np.delete(FFF_Test_3D,np.s_[-10:],axis=1)

# LSTM

In [25]:
from keras import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import SGD
from sklearn.metrics import classification_report
from matplotlib import pyplot
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

In [26]:
Deciles = np.arange(11)
BBB_model_names = ["modelBBB0", "modelBBB1", "modelBBB2", "modelBBB3", "modelBBB4", "modelBBB5",
                   "modelBBB6", "modelBBB7", "modelBBB8", "modelBBB9", "modelBBB10"]
BBB_names = ["BBB0", "BBB1", "BBB2", "BBB3", "BBB4", "BBB5", "BBB6", "BBB7", "BBB8", "BBB9", "BBB10"]
BBB_Train_set = [BBB_Train_3D_Decile0,BBB_Train_3D_Decile1,BBB_Train_3D_Decile2,BBB_Train_3D_Decile3,BBB_Train_3D_Decile4,
                 BBB_Train_3D_Decile5,BBB_Train_3D_Decile6,BBB_Train_3D_Decile7,BBB_Train_3D_Decile8,BBB_Train_3D_Decile9,
                 BBB_Train_3D]
BBB_Test_set = [BBB_Test_3D_Decile0,BBB_Test_3D_Decile1,BBB_Test_3D_Decile2,BBB_Test_3D_Decile3,BBB_Test_3D_Decile4,
                 BBB_Test_3D_Decile5,BBB_Test_3D_Decile6,BBB_Test_3D_Decile7,BBB_Test_3D_Decile8,BBB_Test_3D_Decile9,
                 BBB_Test_3D]

In [ ]:
for i in Deciles:
    
    # Build model for each timestep

    BBB_model_names[i] = Sequential()
    BBB_model_names[i].add(LSTM(32, input_shape=(i+1,3) , activation='sigmoid'))
    BBB_model_names[i].add(Dense(1, activation='sigmoid'))
    BBB_model_names[i].compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    # Fit model for each timestep
    ES = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=0, verbose=0, mode='auto')
    print('Fit LSTM Course BBB - 2 Target Classes - Decile' + str(i))
    history = BBB_model_names[i].fit(BBB_Train_set[i], BBB_Train_Targets_1D, epochs=20, batch_size=1, verbose=1,
                                    validation_data = (BBB_Test_set[i], BBB_Test_Targets_1D), callbacks = [ES])
    
    # plot train vs validation accuracy
    pyplot.plot(history.history['acc'])
    pyplot.plot(history.history['val_acc'])
    pyplot.title('model train vs validation accuracy')
    pyplot.ylabel('accuracy')
    pyplot.xlabel('epoch')
    pyplot.legend(['train', 'validation'], loc='lower right')
    pyplot.show()
    
    # Test model for each timestep
    print('-------------------------------------------')
    print('Test LSTM Course BBB - 2 Target Classes - Decile' + str(i))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=BBB_model_names[i].predict_classes(BBB_Test_set[i]), y_true=BBB_Test_Targets_1D))

## LSTM Course DDD

In [32]:
Deciles = np.arange(11)
DDD_model_names = ["modelDDD0", "modelDDD1", "modelDDD2", "modelDDD3", "modelDDD4", "modelDDD5",
                   "modelDDD6", "modelDDD7", "modelDDD8", "modelDDD9", "modelDDD10"]
DDD_names = ["DDD0", "DDD1", "DDD2", "DDD3", "DDD4", "DDD5", "DDD6", "DDD7", "DDD8", "DDD9", "DDD10"]
DDD_Train_set = [DDD_Train_3D_Decile0,DDD_Train_3D_Decile1,DDD_Train_3D_Decile2,DDD_Train_3D_Decile3,DDD_Train_3D_Decile4,
                 DDD_Train_3D_Decile5,DDD_Train_3D_Decile6,DDD_Train_3D_Decile7,DDD_Train_3D_Decile8,DDD_Train_3D_Decile9,
                 DDD_Train_3D]
DDD_Test_set = [DDD_Test_3D_Decile0,DDD_Test_3D_Decile1,DDD_Test_3D_Decile2,DDD_Test_3D_Decile3,DDD_Test_3D_Decile4,
                 DDD_Test_3D_Decile5,DDD_Test_3D_Decile6,DDD_Test_3D_Decile7,DDD_Test_3D_Decile8,DDD_Test_3D_Decile9,
                 DDD_Test_3D]


In [ ]:
for i in Deciles:
    
    # Build model for each timestep

    DDD_model_names[i] = Sequential()
    DDD_model_names[i].add(LSTM(32, input_shape=(i+1,3) , activation='sigmoid'))
    DDD_model_names[i].add(Dense(1, activation='sigmoid'))
    DDD_model_names[i].compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    # Fit model for each timestep
    ES = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=0, verbose=0, mode='auto')
    print('Fit LSTM Course DDD - 2 Target Classes - Decile' + str(i))
    history = DDD_model_names[i].fit(DDD_Train_set[i], DDD_Train_Targets_1D, epochs=10, batch_size=1, verbose=1,
                                    validation_data = (DDD_Test_set[i], DDD_Test_Targets_1D), callbacks = [ES])
    
    # plot train vs validation accuracy
    pyplot.plot(history.history['acc'])
    pyplot.plot(history.history['val_acc'])
    pyplot.title('model train vs validation accuracy')
    pyplot.ylabel('accuracy')
    pyplot.xlabel('epoch')
    pyplot.legend(['train', 'validation'], loc='lower right')
    pyplot.show()
    
    # Test model for each timestep
    print('-------------------------------------------')
    print('Test LSTM Course DDD - 2 Target Classes - Decile' + str(i))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=DDD_model_names[i].predict_classes(DDD_Test_set[i]), y_true=DDD_Test_Targets_1D))

## LSTM Course FFF

In [33]:
Deciles = np.arange(11)
FFF_model_names = ["modelFFF0", "modelFFF1", "modelFFF2", "modelFFF3", "modelFFF4", "modelFFF5",
                   "modelFFF6", "modelFFF7", "modelFFF8", "modelFFF9", "modelFFF10"]
FFF_names = ["FFF0", "FFF1", "FFF2", "FFF3", "FFF4", "FFF5", "FFF6", "FFF7", "FFF8", "FFF9", "FFF10"]
FFF_Train_set = [FFF_Train_3D_Decile0,FFF_Train_3D_Decile1,FFF_Train_3D_Decile2,FFF_Train_3D_Decile3,FFF_Train_3D_Decile4,
                 FFF_Train_3D_Decile5,FFF_Train_3D_Decile6,FFF_Train_3D_Decile7,FFF_Train_3D_Decile8,FFF_Train_3D_Decile9,
                 FFF_Train_3D]
FFF_Test_set = [FFF_Test_3D_Decile0,FFF_Test_3D_Decile1,FFF_Test_3D_Decile2,FFF_Test_3D_Decile3,FFF_Test_3D_Decile4,
                 FFF_Test_3D_Decile5,FFF_Test_3D_Decile6,FFF_Test_3D_Decile7,FFF_Test_3D_Decile8,FFF_Test_3D_Decile9,
                 FFF_Test_3D]

In [ ]:
for i in Deciles:
    
    # Build model for each timestep

    FFF_model_names[i] = Sequential()
    FFF_model_names[i].add(LSTM(32, input_shape=(i+1,3) , activation='sigmoid'))
    FFF_model_names[i].add(Dense(1, activation='sigmoid'))
    FFF_model_names[i].compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    # Fit model for each timestep
    ES = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=0, verbose=0, mode='auto')
    print('Fit LSTM Course FFF - 2 Target Classes - Decile' + str(i))
    history = FFF_model_names[i].fit(FFF_Train_set[i], FFF_Train_Targets_1D, epochs=10, batch_size=1, verbose=1,
                                    validation_data = (FFF_Test_set[i], FFF_Test_Targets_1D), callbacks = [ES])
    
    # plot train vs validation accuracy
    pyplot.plot(history.history['acc'])
    pyplot.plot(history.history['val_acc'])
    pyplot.title('model train vs validation accuracy')
    pyplot.ylabel('accuracy')
    pyplot.xlabel('epoch')
    pyplot.legend(['train', 'validation'], loc='lower right')
    pyplot.show()
    
    # Test model for each timestep
    print('-------------------------------------------')
    print('Test LSTM Course FFF - 2 Target Classes - Decile' + str(i))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=FFF_model_names[i].predict_classes(FFF_Test_set[i]), y_true=FFF_Test_Targets_1D))

# Alternative approach: model every timestep individually

# LSTM Course BBB

### Train LSTM Course BBB

In [21]:
model.fit(BBB_Train_3D, BBB_Train_Targets_1D, epochs=10, batch_size=1, verbose=1)

Epoch 1/10
3978/3978 [==============================] - 16s 4ms/step - loss: 0.3135 - acc: 0.9065
Epoch 2/10
3978/3978 [==============================] - 16s 4ms/step - loss: 0.3023 - acc: 0.9062
Epoch 3/10
3978/3978 [==============================] - 16s 4ms/step - loss: 0.3006 - acc: 0.9035
Epoch 4/10
3978/3978 [==============================] - 16s 4ms/step - loss: 0.2924 - acc: 0.9045
Epoch 5/10
3978/3978 [==============================] - 16s 4ms/step - loss: 0.3021 - acc: 0.9032
Epoch 6/10
3978/3978 [==============================] - 16s 4ms/step - loss: 0.2943 - acc: 0.9035
Epoch 7/10
3978/3978 [==============================] - 16s 4ms/step - loss: 0.3018 - acc: 0.9052
Epoch 8/10
3978/3978 [==============================] - 15s 4ms/step - loss: 0.3028 - acc: 0.9037
Epoch 9/10
3978/3978 [==============================] - 16s 4ms/step - loss: 0.3052 - acc: 0.9025
Epoch 10/10
3978/3978 [==============================] - 15s 4ms/step - loss: 0.3040 - acc: 0.9055


### Test LSTM Course BBB

In [29]:
model.predict_classes(BBB_Test_3D)

array([[1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
    

In [30]:
print('Classification Report for LSTM predictions:')
print('-------------------------------------------')
print(classification_report(y_pred=model.predict_classes(BBB_Test_3D), y_true=BBB_Test_Targets_1D))

Classification Report for LSTM predictions:
-------------------------------------------
             precision    recall  f1-score   support

          0       0.55      0.92      0.69       391
          1       0.97      0.74      0.84      1152

avg / total       0.86      0.79      0.80      1543



# LSTM Course DDD

### Train LSTM Course DDD

In [32]:
model.predict_classes(DDD_Test_3D)

array([[1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

### Test LSTM Course DDD

In [31]:
model.fit(DDD_Train_3D, DDD_Train_Targets_1D, epochs=10, batch_size=1, verbose=1)

Epoch 1/10
2866/2866 [==============================] - 18s 6ms/step - loss: 0.3181 - acc: 0.8876
Epoch 2/10
2866/2866 [==============================] - 17s 6ms/step - loss: 0.2974 - acc: 0.8901
Epoch 3/10
2866/2866 [==============================] - 17s 6ms/step - loss: 0.2928 - acc: 0.8985
Epoch 4/10
2866/2866 [==============================] - 18s 6ms/step - loss: 0.2941 - acc: 0.9002
Epoch 5/10
2866/2866 [==============================] - 15s 5ms/step - loss: 0.2792 - acc: 0.9013
Epoch 6/10
2866/2866 [==============================] - 16s 5ms/step - loss: 0.2808 - acc: 0.9033
Epoch 7/10
2866/2866 [==============================] - 15s 5ms/step - loss: 0.2690 - acc: 0.9047
Epoch 8/10
2866/2866 [==============================] - 15s 5ms/step - loss: 0.2657 - acc: 0.9030
Epoch 9/10
2866/2866 [==============================] - 16s 6ms/step - loss: 0.2628 - acc: 0.9027
Epoch 10/10
2866/2866 [==============================] - 16s 6ms/step - loss: 0.2584 - acc: 0.9107


In [33]:
print('Classification Report for LSTM predictions:')
print('-------------------------------------------')
print(classification_report(y_pred=model.predict_classes(DDD_Test_3D), y_true=DDD_Test_Targets_1D))

Classification Report for LSTM predictions:
-------------------------------------------
             precision    recall  f1-score   support

          0       0.89      0.71      0.79       364
          1       0.88      0.96      0.92       792

avg / total       0.88      0.88      0.88      1156



# LSTM Course FFF

### Train LSTM Course FFF

In [35]:
model.fit(FFF_Train_3D, FFF_Train_Targets_1D, epochs=10, batch_size=1, verbose=1)

Epoch 1/10
3849/3849 [==============================] - 22s 6ms/step - loss: 0.2619 - acc: 0.9101
Epoch 2/10
3849/3849 [==============================] - 22s 6ms/step - loss: 0.2526 - acc: 0.9174
Epoch 3/10
3849/3849 [==============================] - 23s 6ms/step - loss: 0.2531 - acc: 0.9143
Epoch 4/10
3849/3849 [==============================] - 26s 7ms/step - loss: 0.2470 - acc: 0.9163
Epoch 5/10
3849/3849 [==============================] - 26s 7ms/step - loss: 0.2471 - acc: 0.9163
Epoch 6/10
3849/3849 [==============================] - 25s 7ms/step - loss: 0.2551 - acc: 0.9182
Epoch 7/10
3849/3849 [==============================] - 25s 6ms/step - loss: 0.2560 - acc: 0.9195
Epoch 8/10
3849/3849 [==============================] - 26s 7ms/step - loss: 0.2555 - acc: 0.9174
Epoch 9/10
3849/3849 [==============================] - 25s 7ms/step - loss: 0.2478 - acc: 0.9197
Epoch 10/10
3849/3849 [==============================] - 25s 7ms/step - loss: 0.2531 - acc: 0.9213


### Test LSTM Course FFF

In [36]:
model.predict_classes(FFF_Test_3D)

array([[0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
    

In [37]:
print('Classification Report for LSTM predictions:')
print('-------------------------------------------')
print(classification_report(y_pred=model.predict_classes(FFF_Test_3D), y_true=FFF_Test_Targets_1D))

Classification Report for LSTM predictions:
-------------------------------------------
             precision    recall  f1-score   support

          0       0.93      0.83      0.87       393
          1       0.94      0.98      0.96      1117

avg / total       0.94      0.94      0.94      1510

